In [ ]:
import cmocean as cmo
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import matplotlib.gridspec as gspec
import numpy as np
import seaborn as sns
import scipy.stats
import pandas as pd

import canyon_tools.readout_tools as rout
import canyon_tools.savitzky_golay as sg

import os
import sys
import warnings
warnings.filterwarnings("ignore")

In [ ]:
grid_file1 = '/data/kramosmu/results/TracerExperiments/CNTDIFF/run38/gridGlob.nc'
grid_file2 = '/data/kramosmu/results/TracerExperiments/3DVISC_REALISTIC/run01/gridGlob.nc'
state_file2 = '/data/kramosmu/results/TracerExperiments/3DVISC_REALISTIC/run01/stateGlob.nc'

with Dataset(grid_file1, 'r') as nbl:
    Z1 = nbl.variables['RC'][:]
with Dataset(grid_file2, 'r') as nbl:
    Z2 = nbl.variables['RC'][:]
    hfac = nbl.variables['HFacC'][:,:,:]
with Dataset(state_file2,'r') as stfl:
    time = stfl.variables['T'][:]
    
nx = 616
ny = 360
nz = 90


In [ ]:
class run:
        pass
    
CNTDIFF_run38 = run()
VISC3D_run06 = run()
VISC3D_run05 = run()
VISC3D_run03 = run()
VISC3D_REALISTIC_run01 = run()
VISC3D_REALISTIC_run02 = run()
VISC3D_REALISTIC_run03 = run()
VISC3D_REALISTIC_run04 = run()
VISC3D_REALISTIC_run05 = run()
VISC3D_REALISTIC_run06 = run()

records = [ CNTDIFF_run38 ,
            VISC3D_run06 ,
            VISC3D_run05 ,
            VISC3D_run03 ,
            VISC3D_REALISTIC_run01 ,
            VISC3D_REALISTIC_run03 ,
            VISC3D_REALISTIC_run04 ,
            VISC3D_REALISTIC_run02 ,
            VISC3D_REALISTIC_run05 ,
            VISC3D_REALISTIC_run06 ,
           ]

records_nobase = [ VISC3D_run06 ,
                  VISC3D_run05 ,
                  VISC3D_run03 ,
                  VISC3D_REALISTIC_run01 ,
                  VISC3D_REALISTIC_run03 ,
                  VISC3D_REALISTIC_run04 ,
                  VISC3D_REALISTIC_run02 ,
                  VISC3D_REALISTIC_run05 ,
                  VISC3D_REALISTIC_run06 ,
                 ]
names = ['CNTDIFF_run38', '3DVISC_run06','3DVISC_run05',  '3DVISC_run03',
         '3DVISC_REALISTIC_run01','3DVISC_REALISTIC_run03','3DVISC_REALISTIC_run02',
         '3DVISC_REALISTIC_run04','3DVISC_REALISTIC_run05','3DVISC_REALISTIC_run06']
exps = ['CNTDIFF', '3DVISC', '3DVISC', '3DVISC','3DVISC_REALISTIC', '3DVISC_REALISTIC', 
        '3DVISC_REALISTIC','3DVISC_REALISTIC', '3DVISC_REALISTIC', '3DVISC_REALISTIC']
runs= ['run38', 'run06', 'run05', 'run03', 'run01', 'run03','run02', 'run04', 'run05', 'run06']
labels = [r'base',r'$K_{can}=10^{-2}$ $m^2$s$^{-1}$',r'$K_{can}=5\times10^{-2}$ $m^2$s$^{-1}$',
          '$K_{can}=10^{-3}$ $m^2$s$^{-1}$', '$K_{Mty}$', '$K_{Mty} rim$',
          '$K_{Eel}$', '$K_{Eel}$ rim','$K_{Asc}$', '$K_{Asc}$ rim']
colors = ['black', 'kelley green','leaf green', 'olive', 'cerulean','sky blue','purple','orchid','cherry', 'salmon']
Zs = [Z1, Z1, Z1,Z1, Z2, Z2, Z2,Z2, Z2, Z2]


for rec, name, exp, runnum, col, lab, Z in zip(records,names,exps,runs,colors,labels, Zs):
    rec.name = name
    rec.exp = exp
    rec.run = runnum
    rec.color = col
    rec.label = lab
    rec.Z = Z

In [ ]:
kv_dir = '/ocean/kramosmu/Building_canyon/BuildCanyon/Stratification/616x360x90/'
ini_kv_files = [kv_dir + 'KrDiff1E_5_1E_2_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff1E_5_5E_3_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff1E_5_1E_3_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff_Mty_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff_Mty_rim_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff_Eel_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff_Eel_rim_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff_Asc_90zlev_616x360_Quad.bin',
                kv_dir + 'KrDiff_Asc_rim_90zlev_616x360_Quad.bin']

dt = np.dtype('>f8')  # float 64 big endian
DnS = [245, 200] # y, x indices of DnS station

ini_kv_profiles = np.zeros((len(ini_kv_files),nz))

for file, ii in zip(ini_kv_files, range(len(ini_kv_files))):
    data = np.fromfile(file, dt)
    print(file)
    ini_kv = np.reshape(data,(nz,ny,nx),order='C')
    ini_kv_profiles[ii,:] = ini_kv[:, DnS[0], DnS[1]] 